In [1]:
%pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html
%pip install labml-nn

Looking in links: https://data.dgl.ai/wheels/torch-2.3/repo.html
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pydantic


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F


class LinearNeuralNetwork(nn.Module):
    def __init__(self, nfeat, nclass, bias=True):
        super(LinearNeuralNetwork, self).__init__()
        self.W = nn.Linear(nfeat, nclass, bias=bias)

    def forward(self, x):
        return self.W(x)


def symmetric_normalize_adjacency(graph):
    """Symmetric normalize graph adjacency matrix."""
    indices = torch.stack(graph.edges())
    n = graph.num_nodes()
    adj = dglsp.spmatrix(indices, shape=(n, n))
    deg_invsqrt = dglsp.diag(adj.sum(0)) ** -0.5
    return deg_invsqrt @ adj @ deg_invsqrt


def model_test(model, embeds):
    model.eval()
    with torch.no_grad():
        output = model(embeds)
        pred = output.argmax(dim=-1)
        test_mask, tv_mask = model.test_mask, model.tv_mask
        loss_tv = F.mse_loss(output[tv_mask], model.label_one_hot[tv_mask])
    accs = []
    for mask in [tv_mask, test_mask]:
        accs.append(
            float((pred[mask] == model.label[mask]).sum() / mask.sum()))
    return loss_tv.item(), accs[0], accs[1], pred

c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import dgl.sparse as dglsp
import torch.nn as nn
import torch.nn.functional as F


class OGC(nn.Module):
    def __init__(self, graph):
        super(OGC, self).__init__()
        self.linear_clf = LinearNeuralNetwork(
            nfeat=graph.ndata["feat"].shape[1],
            nclass=graph.ndata["label"].max().item() + 1,
            bias=False,
        )

        self.label = graph.ndata["label"]
        self.label_one_hot = F.one_hot(graph.ndata["label"]).float()
        # LIM trick, else use both train and val set to construct this matrix.
        self.label_idx_mat = dglsp.diag(graph.ndata["train_mask"]).float()

        self.test_mask = graph.ndata["test_mask"]
        self.tv_mask = graph.ndata["train_mask"] + graph.ndata["val_mask"]

    def forward(self, x):
        return self.linear_clf(x)

    def update_embeds(self, embeds, lazy_adj, args):
        """Update classifier's weight by training a linear supervised model."""
        pred_label = self(embeds).data
        clf_weight = self.linear_clf.W.weight.data

        # Update the smoothness loss via LGC.
        embeds = dglsp.spmm(lazy_adj, embeds)

        # Update the supervised loss via SEB.
        deriv_sup = 2 * dglsp.matmul(
            dglsp.spmm(self.label_idx_mat, -self.label_one_hot + pred_label),
            clf_weight,
        )
        embeds = embeds - args.lr_sup * deriv_sup

        args.lr_sup = args.lr_sup * args.decline
        return embeds

In [5]:
import argparse
import time

import dgl.sparse as dglsp

import torch.nn.functional as F
import torch.optim as optim
from dgl import AddSelfLoop
from dgl.data import PubmedGraphDataset

parser = argparse.ArgumentParser()
args = parser.parse_args([])
args.dataset = "pubmed"
args.decline = 0.9
args.lr_sup = 0.001
args.lr_clf = 0.5
args.beta = 0.1
args.max_sim_rate = 0.995
args.max_patience = 2
args.device = torch.device("cpu")

In [6]:
def train(model, embeds, lazy_adj, args):
    patience = 0
    _, _, last_acc, last_output = model_test(model, embeds)

    tv_mask = model.tv_mask
    optimizer = optim.SGD(model.parameters(), lr=args.lr_clf)

    for i in range(64):
        model.train()
        output = model(embeds)
        loss_tv = F.mse_loss(
            output[tv_mask], model.label_one_hot[tv_mask], reduction="sum"
        )
        optimizer.zero_grad()
        loss_tv.backward()
        optimizer.step()

        # Updating node embeds by LGC and SEB jointly.
        embeds = model.update_embeds(embeds, lazy_adj, args)

        loss_tv, acc_tv, acc_test, pred = model_test(model, embeds)
        print(
            "epoch {} loss_tv {:.4f} acc_tv {:.4f} acc_test {:.4f}".format(
                i + 1, loss_tv, acc_tv, acc_test
            )
        )

        sim_rate = float(int((pred == last_output).sum()) / int(pred.shape[0]))
        if sim_rate > args.max_sim_rate:
            patience += 1
            if patience > args.max_patience:
                break
        last_acc = acc_test
        last_output = pred
    return last_acc

In [7]:
!nvidia-smi

Tue Dec 31 19:07:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P3              11W /  30W |    144MiB /  4096MiB |     32%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
transform = AddSelfLoop()
data = PubmedGraphDataset(transform=transform)
graph = data[0].to(args.device)

C:\Users\Taneem\.dgl\pubmed.zip: 100%|██████████| 4.93M/4.93M [00:09<00:00, 522kB/s] 


Extracting file to C:\Users\Taneem\.dgl\pubmed_35464cad
Finished data loading and preprocessing.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [9]:
graph.has_edges_between([0], [1])

tensor([False])

In [10]:
def pagerank(net, weights={}, q=0.5, eps=0.01, maxIters=500, verbose=False, weightName='weight'):

    incomingTeleProb = {}
    prevVisitProb = {}
    currVisitProb = {}
    N = net.number_of_nodes()

    totWeight = sum([w for v, w in weights.items()])

    if totWeight == 0:
        incomingTeleProb = dict.fromkeys(net, 1.0 / N)
        prevVisitProb = incomingTeleProb.copy()
        currVisitProb = incomingTeleProb.copy()
    else:
        minPosWeight = 1.0
        for v, weight in weights.items():
            if weight == 0:
                continue
            minPosWeight = min(minPosWeight, 1.0 * weight / totWeight)

        smallWeight = minPosWeight / (10 ** 6)

        for v in net.nodes():
            weight = weights.get(v, 0.0)
            incomingTeleProb[v] = 1.0 * \
                (weight + smallWeight) / (totWeight + smallWeight * N)
        prevVisitProb = incomingTeleProb.copy()
        currVisitProb = incomingTeleProb.copy()

    outDeg = {}
    zeroDegNodes = set()
    for v in net.nodes():
        outDeg[v] = 1.0 * net.out_degree(v, weight=weightName)
        if outDeg[v] == 0:
            zeroDegNodes.add(v)

    iters = 0
    finished = False
    while not finished:
        iters += 1
        prevVisitProb = currVisitProb.copy()
        maxDiff = 0

        zSum = sum([prevVisitProb[x] for x in zeroDegNodes]) / N

        for v in net.nodes():
            eSum = 0
            for u in net.predecessors(v):
                # Handle missing 'weight' attribute by assuming it's 1.0
                w_uv = 1.0 * net[u][v].get(weightName, 1.0)
                eSum += w_uv / outDeg[u] * prevVisitProb[u]

            currVisitProb[v] = q * incomingTeleProb[v] + \
                (1 - q) * (eSum + zSum)
            maxDiff = max(maxDiff, abs(
                (prevVisitProb[v] - currVisitProb[v]) / currVisitProb[v]))

        if verbose:
            print('\tIteration %d, max difference %f' % (iters, maxDiff))
            if maxDiff < eps:
                print('PageRank converged after %d iterations, max difference %f.' % (
                    iters, maxDiff))

        if iters >= maxIters:
            print(
                'WARNING: PageRank terminated because max iters (%d) was reached.' % (maxIters))

        finished = (maxDiff < eps) or (iters >= maxIters)

    return currVisitProb

In [11]:
import networkx as nx

# Convert DGL graph to NetworkX
nx_graph = graph.to_networkx()

# Run PageRank
pagerank_scores = pagerank(nx_graph)

# Assuming graph.ndata['feat'] or similar contains node features, map DGL nodes to NetworkX nodes
# If the DGL graph uses integer indices for nodes, you can map directly

# Print pagerank scores (optional)
print(pagerank_scores)

{0: 4.313242539159024e-05, 1: 4.34675143192645e-05, 2: 4.276619825721096e-05, 3: 3.7387378559985726e-05, 4: 3.7693398099297213e-05, 5: 3.798142469130875e-05, 6: 8.105295123286397e-05, 7: 9.387410575367942e-05, 8: 3.9208839275377475e-05, 9: 6.431537449796779e-05, 10: 4.836000235052288e-05, 11: 3.907813926708187e-05, 12: 0.00011196173907463303, 13: 3.617699520658647e-05, 14: 3.799576862995219e-05, 15: 4.799267874766282e-05, 16: 0.00024251357055732684, 17: 8.318238294555078e-05, 18: 5.3624311256241145e-05, 19: 5.846661772321927e-05, 20: 3.84988718881687e-05, 21: 4.562662555005514e-05, 22: 3.5731532611520074e-05, 23: 3.905521609346174e-05, 24: 3.8440107340394925e-05, 25: 4.045727223668179e-05, 26: 6.389357991266173e-05, 27: 4.477490095683654e-05, 28: 3.890489807983709e-05, 29: 3.7165713419323496e-05, 30: 3.890957997292185e-05, 31: 3.831490759711056e-05, 32: 3.996610258200284e-05, 33: 4.126774690506562e-05, 34: 3.9398648845688e-05, 35: 0.0001406109017942284, 36: 3.8281058480598004e-05, 37: 

In [12]:
import torch as th

features = graph.ndata["feat"]
adj = symmetric_normalize_adjacency(graph)
# I_N = dglsp.identity((features.shape[0], features.shape[0]))
# print(I_N)

# Lazy random walk (also known as lazy graph convolution).
# lazy_adj = dglsp.add((1 - args.beta) * I_N, args.beta * adj).to(args.device)
# print(lazy_adj)

# Convert PageRank scores to a tensor, matching DGL graph node indices with NetworkX nodes
pagerank_tensor = th.tensor(
    [pagerank_scores[int(node.item())] for node in graph.nodes()])

# Normalize PageRank scores (optional)
pagerank_tensor /= pagerank_tensor.sum()

print(pagerank_tensor)

# Assuming pagerank_tensor has been defined earlier and is on the correct device
# Create a sparse diagonal matrix from the PageRank scores
num_nodes = pagerank_tensor.shape[0]  # Get the number of nodes
pagerank_diag = dglsp.diag(pagerank_tensor).to(
    args.device)  # Convert to a DGL SparseMatrix

# Now, modify lazy_adj by incorporating PageRank scores
# Ensure adj is a DGL SparseMatrix as well
lazy_adj = dglsp.add((1 - args.beta) * pagerank_diag, args.beta * adj)

print(lazy_adj)  # Check the lazy adjacency matrix

tensor([4.3132e-05, 4.3468e-05, 4.2766e-05,  ..., 3.8813e-05, 3.8867e-05,
        4.1531e-05])
SparseMatrix(indices=tensor([[    0,     0,     0,  ..., 19715, 19716, 19716],
                             [    0,  1378,  1544,  ..., 19715, 16030, 19716]]),
             values=tensor([0.0167, 0.0067, 0.0068,  ..., 0.0500, 0.0250, 0.0500]),
             shape=(19717, 19717), nnz=108365)


In [13]:
model = OGC(graph).to(args.device)

In [14]:
sum(p.numel() for p in model.parameters())

1500

In [15]:
start_time = time.time()
res = train(model, features, lazy_adj, args)
time_tot = time.time() - start_time

print(f"Test Acc:{res:.4f}")
print(f"Total Time:{time_tot:.4f}")

epoch 1 loss_tv 0.3038 acc_tv 0.8518 acc_test 0.7940
epoch 2 loss_tv 0.3156 acc_tv 0.8161 acc_test 0.7980
epoch 3 loss_tv 0.3250 acc_tv 0.8411 acc_test 0.7920
epoch 4 loss_tv 0.3220 acc_tv 0.8321 acc_test 0.7850
epoch 5 loss_tv 0.3241 acc_tv 0.8268 acc_test 0.7810
epoch 6 loss_tv 0.3238 acc_tv 0.8107 acc_test 0.7770
epoch 7 loss_tv 0.3245 acc_tv 0.7964 acc_test 0.7480
epoch 8 loss_tv 0.3249 acc_tv 0.7643 acc_test 0.7280
epoch 9 loss_tv 0.3254 acc_tv 0.7518 acc_test 0.7090
epoch 10 loss_tv 0.3258 acc_tv 0.7232 acc_test 0.6910
epoch 11 loss_tv 0.3263 acc_tv 0.7161 acc_test 0.6830
epoch 12 loss_tv 0.3267 acc_tv 0.7125 acc_test 0.6780
epoch 13 loss_tv 0.3272 acc_tv 0.7107 acc_test 0.6770
epoch 14 loss_tv 0.3276 acc_tv 0.7125 acc_test 0.6770
epoch 15 loss_tv 0.3280 acc_tv 0.7125 acc_test 0.6770
Test Acc:0.6770
Total Time:6.7917
